### Convert spike timepoints index



In [2]:
import os
import pandas as pd
import numpy as np
import spikeinterface as si

# move to project path
PROJ_PATH = "/gpfs/bbp.cscs.ch/project/proj68/home/laquitai/spikebias/"
os.chdir(PROJ_PATH)

from src.nodes.utils import get_config
from src.nodes.truth.silico import ground_truth

EXPERIMENT = "silico_horvath"
RUN = "concatenated/probe_2"
data_conf, param_conf = get_config(EXPERIMENT, RUN).values()

gt = data_conf["sorting"]["simulation"]["ground_truth"]["output"]

2023-12-01 11:57:35,607 - root - utils.py - get_config - INFO - Reading experiment config.
2023-12-01 11:57:35,639 - root - utils.py - get_config - INFO - Reading experiment config. - done


- index is in milliseconds

In [3]:
# get read paths
SPIKE_FILE_PATH = data_conf["dataeng"]["campaign"]["output"]["spike_file_path"]
LFP_FILE_PATH = data_conf["dataeng"]["campaign"]["output"]["trace_file_path"]

# get spikes and lfp
spike = pd.read_pickle(SPIKE_FILE_PATH)
lfp = pd.read_pickle(LFP_FILE_PATH)
truth = si.load_extractor(gt)

In [4]:
# takes ...

# carefully deal with numerical precision issues
SPIKE_SAMP_FREQ = 40000
TRACE_SAMP_FREQ = 20000
SPIKE_SFREQ_MS = SPIKE_SAMP_FREQ / 1000
TRACE_SFREQ_MS = TRACE_SAMP_FREQ / 1000

# get number of timepoints on spike index reference
spike_npoint_for_40KHz = spike.index.values * SPIKE_SFREQ_MS

# get number of timepoints on trace index reference
conv_factor = TRACE_SFREQ_MS / SPIKE_SFREQ_MS
spike_tpoints_for_20KHz = (spike_npoint_for_40KHz * conv_factor).astype(int)

# narrow the search space for each spike
spike_loc = []
for s_i, spike_ms_i in enumerate(spike.index):
    # define narrower search window
    start_wind = spike_tpoints_for_20KHz[s_i] - 30
    end_wind = spike_tpoints_for_20KHz[s_i] + 30

    # for the last spike,  end window at lfp's last timepoint
    if s_i == spike.index.shape[0]:
        end_wind = lfp.shape[0]
    search_window = np.arange(start_wind, end_wind, 1)

    # append spike timepoints
    loc_in_window = np.abs(lfp.index[search_window] - spike_ms_i).argmin()
    spike_loc.append(start_wind + loc_in_window)

In [7]:
print(end_wind)

23519946

In [458]:
# unit testing
np.array_equal(np.array(spike_loc), truth.get_all_spike_trains()[0][0])

True

### Implemented as module in src

In [462]:
from src.nodes.load import load_campaign_params

simulation = load_campaign_params(data_conf)
truth = ground_truth.load_spike_fast(simulation, data_conf, param_conf)

/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/3_sfn_2023/envs/npx_10m_384ch_unit_classes/lib/python3.9/site-packages/pandas/core/indexes/multi.py:643: DeprecationWarning: `cumproduct` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `cumprod` instead.
  codes = cartesian_product(codes)
/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/3_sfn_2023/envs/npx_10m_384ch_unit_classes/lib/python3.9/site-packages/pandas/core/reshape/util.py:60: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  return [
/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/3_sfn_2023/envs/npx_10m_384ch_unit_classes/lib/python3.9/site-packages/pandas/core/reshape/util.py:60: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  return [
/gpfs/bbp.cscs.ch/project/proj68/scratch/laquitai/3_sfn_2023/envs/npx_10m_384ch_unit_classes/lib/pyth

2023-11-30 19:10:08,480 - root - ground_truth.py - create_sorting_object - INFO - Creating SpikeInterface's SortingTrue extractor


In [464]:
# unit testing
np.array_equal(np.array(spike_loc), truth.get_all_spike_trains()[0][0])

True